# Section: Encrypted Deep Learning

- Lesson: Reviewing Additive Secret Sharing
- Lesson: Encrypted Subtraction and Public/Scalar Multiplication
- Lesson: Encrypted Computation in PySyft
- Project: Build an Encrypted Database
- Lesson: Encrypted Deep Learning in PyTorch
- Lesson: Encrypted Deep Learning in Keras
- Final Project

# Lesson: Reviewing Additive Secret Sharing

_For more great information about SMPC protocols like this one, visit https://mortendahl.github.io. With permission, Morten's work directly inspired this first teaching segment._

In [1]:
import random
import numpy as np

BASE = 10

PRECISION_INTEGRAL = 8
PRECISION_FRACTIONAL = 8
Q = 293973345475167247070445277780365744413

PRECISION = PRECISION_INTEGRAL + PRECISION_FRACTIONAL

assert(Q > BASE**PRECISION)

def encode(rational):
    upscaled = int(rational * BASE**PRECISION_FRACTIONAL)
    field_element = upscaled % Q
    return field_element

def decode(field_element):
    upscaled = field_element if field_element <= Q/2 else field_element - Q
    rational = upscaled / BASE**PRECISION_FRACTIONAL
    return rational

def encrypt(secret):
    first  = random.randrange(Q)
    second = random.randrange(Q)
    third  = (secret - first - second) % Q
    return [first, second, third]

def decrypt(sharing):
    return sum(sharing) % Q

def add(a, b):
    c = list()
    for i in range(len(a)):
        c.append((a[i] + b[i]) % Q)
    return tuple(c)

In [2]:
x = encrypt(encode(5.5))
x

[200586597334110239648081182858842528589,
 204926951123661908084131222454606661440,
 182433142492562346408678150247832298797]

In [3]:
y = encrypt(encode(2.3))
y

[205360498566252727056817627142307939734,
 215983483300439481912404806303402943740,
 166602709083642285171668122115250605351]

In [4]:
z = add(x,y)
z

(111973750425195719634453532220784723910,
 126937088948934142926090750977643860767,
 55062506101037384509900994582717159735)

In [5]:
decode(decrypt(z))

7.79999999

# Lesson: Encrypted Subtraction and Public/Scalar Multiplication

In [6]:
field = 23740629843760239486723

In [7]:
x = 5

bob_x_share = 2372385723 # random number
alices_x_share = field - bob_x_share + x

In [8]:
(bob_x_share + alices_x_share) % field

5

In [9]:
field = 10

x = 5

bob_x_share = 8
alice_x_share = field - bob_x_share + x

y = 1

bob_y_share = 9
alice_y_share = field - bob_y_share + y

In [10]:
((bob_x_share + alice_x_share) - (bob_y_share + alice_y_share)) % field

4

In [11]:
((bob_x_share - bob_y_share) + (alice_x_share - alice_y_share)) % field

4

In [12]:
bob_x_share + alice_x_share + bob_y_share + alice_y_share

26

In [13]:
bob_z_share = (bob_x_share - bob_y_share)
alice_z_share = (alice_x_share - alice_y_share)

In [14]:
(bob_z_share + alice_z_share) % field

4

In [15]:
def sub(a, b):
    c = list()
    for i in range(len(a)):
        c.append((a[i] - b[i]) % Q)
    return tuple(c)

In [16]:
field = 10

x = 5

bob_x_share = 8
alice_x_share = field - bob_x_share + x

y = 1

bob_y_share = 9
alice_y_share = field - bob_y_share + y

In [17]:
bob_x_share + alice_x_share

15

In [18]:
bob_y_share + alice_y_share

11

In [19]:
((bob_y_share * 3) + (alice_y_share * 3)) % field

3

In [20]:
def imul(a, scalar):
    
    # logic here which can multiply by a public scalar
    
    c = list()
    
    for i in range(len(a)):
        c.append((a[i] * scalar) % Q)
        
    return tuple(c)

In [21]:
x = encrypt(encode(5.5))
x

[35588733695667958677668760486088147387,
 156775780026187286915441394721797619641,
 101608831753312001477335122573029977385]

In [22]:
z = imul(x, 3)

In [23]:
decode(decrypt(z))

16.5

# Lesson: Encrypted Computation in PySyft

In [1]:
import syft as sy
import torch as th
hook = sy.TorchHook(th)
from torch import nn, optim

W0805 16:23:11.326478 14196 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was 'd:\ProgramData\Miniconda3\envs\pysyft\lib\site-packages\tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0805 16:23:11.341447 14196 deprecation_wrapper.py:119] From d:\ProgramData\Miniconda3\envs\pysyft\lib\site-packages\tf_encrypted\session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [2]:
bob = sy.VirtualWorker(hook, id="bob").add_worker(sy.local_worker)
alice = sy.VirtualWorker(hook, id="alice").add_worker(sy.local_worker)
secure_worker = sy.VirtualWorker(hook, id="secure_worker").add_worker(sy.local_worker)

In [3]:
x = th.tensor([1,2,3,4])
y = th.tensor([2,-1,1,0])

In [4]:
x = x.share(bob, alice, crypto_provider=secure_worker)

In [5]:
y = y.share(bob, alice, crypto_provider=secure_worker)

In [6]:
z = x + y
z.get()

tensor([3, 1, 4, 4])

In [7]:
z = x - y
z.get()

tensor([-1,  3,  2,  4])

In [8]:
z = x * y
z.get()

tensor([ 2, -2,  3,  0])

In [9]:
z = x > y
z.get()

tensor([0, 1, 1, 1])

In [10]:
z = x < y
z.get()

tensor([1, 0, 0, 0])

In [11]:
z = x == y
z.get()

tensor([0, 0, 0, 0])

In [12]:
x = th.tensor([1,2,3,4])
y = th.tensor([2,-1,1,0])

x = x.fix_precision().share(bob, alice, crypto_provider=secure_worker)
y = y.fix_precision().share(bob, alice, crypto_provider=secure_worker)

In [13]:
z = x + y
z.get().float_precision()

tensor([3., 1., 4., 4.])

In [14]:
z = x - y
z.get().float_precision()

tensor([-1.,  3.,  2.,  4.])

In [15]:
z = x * y
z.get().float_precision()

tensor([ 2., -2.,  3.,  0.])

In [16]:
z = x > y
z.get().float_precision()

tensor([0., 1., 1., 1.])

In [17]:
z = x < y
z.get().float_precision()

tensor([1., 0., 0., 0.])

In [18]:
z = x == y
z.get().float_precision()

tensor([0., 0., 0., 0.])

# Project: Build an Encrypted Database

In [12]:
import syft as sy
import torch as th
hook = sy.TorchHook(th)

W0808 19:20:07.603293 11664 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was 'd:\ProgramData\Miniconda3\envs\pysyft\lib\site-packages\tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0808 19:20:07.621282 11664 deprecation_wrapper.py:119] From d:\ProgramData\Miniconda3\envs\pysyft\lib\site-packages\tf_encrypted\session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [13]:
import string

In [14]:
char2index = {}
index2char = {}

In [15]:
for i,char in enumerate(' ' + string.ascii_lowercase + '0123456789' + string.punctuation):
    char2index[char] = i
    index2char[i] = char

In [16]:
str_input = "Hello"
max_len = 8

In [17]:
def string2values(str_input, max_len=8):

    str_input = str_input[:max_len].lower()

    # pad strings shorter than max len
    if(len(str_input) < max_len):
        str_input = str_input + "." * (max_len - len(str_input))

    values = list()
    for char in str_input:
        values.append(char2index[char])

    return th.tensor(values).long()

In [18]:
def values2string(input_values):
    s = ""
    for value in input_values:
        s += index2char[int(value)]
    return s

In [19]:
def strings_equal(str_a, str_b):

    vect = (str_a * str_b).sum(1)

    x = vect[0]

    for i in range(vect.shape[0] - 1):
        x = x * vect[i + 1]    

    return x

In [20]:
def one_hot(index, length):
    vect = th.zeros(length).long()
    vect[index] = 1
    return vect

In [21]:
def string2one_hot_matrix(str_input, max_len=8):

    str_input = str_input[:max_len].lower()

    # pad strings shorter than max len
    if(len(str_input) < max_len):
        str_input = str_input + "." * (max_len - len(str_input))

    char_vectors = list()
    for char in str_input:
        char_v = one_hot(char2index[char], len(char2index)).unsqueeze(0)
        char_vectors.append(char_v)
        
    return th.cat(char_vectors, dim=0)

In [22]:
class EncryptedDB():
    
    def __init__(self, *owners, max_key_len=8, max_val_len=8):
        self.max_key_len = max_key_len
        self.max_val_len = max_val_len
        
        self.keys = list()
        self.values = list()
        self.owners = owners
        
    def add_entry(self, key, value):
        key = string2one_hot_matrix(key)
        key = key.share(*self.owners)
        self.keys.append(key)
        
        value = string2values(value, max_len=self.max_val_len)
        value = value.share(*self.owners)
        self.values.append(value)
        
    def query(self, query_str):
        query_matrix = string2one_hot_matrix(query_str)
        
        query_matrix = query_matrix.share(*self.owners)

        key_matches = list()
        for key in self.keys:

            key_match = strings_equal(key, query_matrix)
            key_matches.append(key_match)

        result = self.values[0] * key_matches[0]

        for i in range(len(self.values) - 1):
            result += self.values[i+1] * key_matches[i+1]
            
        result = result.get()

        return values2string(result).replace(".","")

In [24]:
bob = sy.VirtualWorker(hook, id="bob").add_worker(sy.local_worker)
alice = sy.VirtualWorker(hook, id="alice").add_worker(sy.local_worker)
secure_worker = sy.VirtualWorker(hook, id="secure_worker").add_worker(sy.local_worker)

In [25]:
db = EncryptedDB(bob, alice, secure_worker, max_val_len=256)

db.add_entry("Bob","(123) 456 7890")
db.add_entry("Bill", "(234) 567 8901")
db.add_entry("Sam","(345) 678 9012")
db.add_entry("Key","really big json value")

db.query("Bob")

'(123) 456 7890'

# Lesson: Encrypted Deep Learning in PyTorch

### Train a Model

In [26]:
from torch import nn
from torch import optim
import torch.nn.functional as F

# A Toy Dataset
data = th.tensor([[0,0],[0,1],[1,0],[1,1.]], requires_grad=True)
target = th.tensor([[0],[0],[1],[1.]], requires_grad=True)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2, 20)
        self.fc2 = nn.Linear(20, 1)

    def forward(self, x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x

# A Toy Model
model = Net()

def train():
    # Training Logic
    opt = optim.SGD(params=model.parameters(),lr=0.1)
    for iter in range(20):

        # 1) erase previous gradients (if they exist)
        opt.zero_grad()

        # 2) make a prediction
        pred = model(data)

        # 3) calculate how much we missed
        loss = ((pred - target)**2).sum()

        # 4) figure out which weights caused us to miss
        loss.backward()

        # 5) change those weights
        opt.step()

        # 6) print our progress
        print(loss.data)
        
train()

tensor(2.0110)
tensor(21.1033)
tensor(26.1786)
tensor(1.1621)
tensor(1.0021)
tensor(0.9826)
tensor(0.9669)
tensor(0.9495)
tensor(0.9298)
tensor(0.9071)
tensor(0.8810)
tensor(0.8515)
tensor(0.8187)
tensor(0.7833)
tensor(0.7464)
tensor(0.6959)
tensor(0.5684)
tensor(0.4497)
tensor(0.3375)
tensor(0.2470)


In [27]:
model(data)

tensor([[0.2994],
        [0.1267],
        [0.9939],
        [0.7340]], grad_fn=<AddmmBackward>)

## Encrypt the Model and Data

In [28]:
encrypted_model = model.fix_precision().share(alice, bob, crypto_provider=secure_worker)

In [29]:
list(encrypted_model.parameters())

[Parameter containing:
 Parameter>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
 	-> (Wrapper)>[PointerTensor | me:92275597080 -> alice:32510194076]
 	-> (Wrapper)>[PointerTensor | me:36414028130 -> bob:17445668688]
 	*crypto provider: secure_worker*, Parameter containing:
 Parameter>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
 	-> (Wrapper)>[PointerTensor | me:70402071373 -> alice:73411591940]
 	-> (Wrapper)>[PointerTensor | me:88336954905 -> bob:2251328014]
 	*crypto provider: secure_worker*, Parameter containing:
 Parameter>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
 	-> (Wrapper)>[PointerTensor | me:16866731723 -> alice:23243067845]
 	-> (Wrapper)>[PointerTensor | me:59032477396 -> bob:3071098287]
 	*crypto provider: secure_worker*, Parameter containing:
 Parameter>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
 	-> (Wrapper)>[PointerTensor | me:89609210364 -> alice:87948600244]
 	-> (Wrapper)>[PointerTensor | me:70089696863 -> bob:5068675156

In [30]:
encrypted_data = data.fix_precision().share(alice, bob, crypto_provider=secure_worker)

In [31]:
encrypted_data

(Wrapper)>FixedPrecisionTensor>(Wrapper)>[AdditiveSharingTensor]
	-> (Wrapper)>[PointerTensor | me:12720213727 -> alice:85057267172]
	-> (Wrapper)>[PointerTensor | me:98754464501 -> bob:64487884545]
	*crypto provider: secure_worker*

In [32]:
encrypted_prediction = encrypted_model(encrypted_data)

In [33]:
encrypted_prediction.get().float_precision()

tensor([[0.2980],
        [0.1270],
        [0.9920],
        [0.7320]])